
# Análise de Preferências de Estudantes
Este notebook realiza uma análise exploratória de dados sobre as preferências de estudantes, com base em um conjunto de dados fornecido. O processo inclui o carregamento dos dados, inspeção inicial, limpeza e preparação, seguido pela visualização de diversos insights sobre as preferências dos alunos e outras características relevantes.




## 1. Importação de Bibliotecas
Importação das bibliotecas necessárias para manipulação de dados `(pandas)` e visualização `(matplotlib.pyplot e seaborn)`.



In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

ModuleNotFoundError: No module named 'pandas'

## 2. Carregamento dos Dados
Carregamento do conjunto de dados `student_preferences_extended.csv` a partir de uma URL pública no GitHub.



In [ ]:
url = "https://raw.githubusercontent.com/puc-tech/challenge/refs/heads/main/student_preferences_extended.csv"
df = pd.read_csv(url)

## 3. Inspeção Inicial dos Dados
Nesta etapa, realizamos uma verificação inicial do DataFrame para entender sua estrutura, as primeiras linhas, informações gerais sobre os tipos de dados e a contagem de valores ausentes por coluna. Também obtemos estatísticas descritivas.



In [ ]:
print("--- Inspeção Inicial dos Dados ---")
print("\nPrimeiras 5 linhas do DataFrame:")
print(df.head())

print("\nInformações gerais do DataFrame:")
df.info()

print("\nEstatísticas descritivas (incluindo colunas categóricas):")
print(df.describe(include='all'))

print("\nContagem de valores ausentes por coluna:")
print(df.isnull().sum())

## 4. Limpeza e Preparação dos Dados
Esta seção foca na limpeza e preparação do `DataFrame` para análise. As etapas incluem:

 - Remoção de colunas consideradas metadados ou que exigiriam processamento de linguagem natural complexo para esta análise (`tempo_resposta`, `comentario`).
 - Remoção de linhas com valores ausentes em colunas chave para a análise de preferências (`linguagem_preferida`, `horario_estudo`, `formato_conteudo_principal`).
 - Preenchimento de valores ausentes em colunas numéricas com a mediana.
 - Preenchimento de valores ausentes em colunas categóricas com "Não Informado" ou a moda.
 - Tratamento e tentativa de conversão de colunas para o tipo booleano.


In [ ]:
print("\n--- Limpeza e Preparação dos Dados ---")

# Remover colunas que geralmente não são usadas diretamente em análises quantitativas de preferência agregada
# ou são metadados.
cols_to_drop_initial = ['tempo_resposta', 'comentario']
df_cleaned = df.drop(columns=[col for col in cols_to_drop_initial if col in df.columns])

# Para as colunas chave das análises de preferência solicitadas,
# vamos remover linhas onde esses dados estão ausentes para garantir a precisão dessas análises específicas.
key_preference_cols = ['linguagem_preferida', 'horario_estudo', 'formato_conteudo_principal']
df_cleaned.dropna(subset=key_preference_cols, inplace=True)
print(f"\nShape do DataFrame após remover NaNs das colunas chave de preferência: {df_cleaned.shape}")

# Para outras colunas numéricas que podem ser usadas em gráficos,
# podemos preencher NaNs com a média ou mediana.
numeric_cols_to_fill = ['horas_estudo_dia', 'media_geral', 'satisfacao_curso', 'faltas_percentual', 'idade', 'semestre']
for col in numeric_cols_to_fill:
    if col in df_cleaned.columns:
        df_cleaned[col].fillna(df_cleaned[col].median(), inplace=True) # Usando mediana por ser menos sensível a outliers

# Para outras colunas categóricas, podemos preencher com 'Não Informado' ou a moda.
categorical_cols_to_fill = ['framework_preferido', 'formato_conteudo_secundario',
                              'ambiente_desenvolvimento', 'sistema_operacional', 'area_interesse']
for col in categorical_cols_to_fill:
    if col in df_cleaned.columns:
        df_cleaned[col].fillna('Não Informado', inplace=True)

# Booleanos: preencher com False (ou a moda) se fizer sentido contextual.
boolean_cols = ['estuda_em_grupo', 'usa_biblioteca', 'participa_monitoria', 'busca_estagio', 'prefere_backend', 'interesse_pesquisa']
for col in boolean_cols:
    if col in df_cleaned.columns:
        if df_cleaned[col].isnull().any():
            df_cleaned[col].fillna(df_cleaned[col].mode()[0], inplace=True) # Preenche com a moda
        # Tentativa de converter para tipo booleano de forma segura
        try:
            if not pd.api.types.is_bool_dtype(df_cleaned[col]):
                if df_cleaned[col].dtype == 'object':
                    map_dict = {'True': True, 'False': False, True: True, False: False, 'Sim': True, 'Não': False} # Expandido
                    # Aplicar o mapa apenas para valores que existem no mapa, outros podem virar a moda ou False
                    default_bool_value = False # Ou df_cleaned[col].mode()[0] se quiser a moda como padrão
                    df_cleaned[col] = df_cleaned[col].map(lambda x: map_dict.get(x, map_dict.get(str(x), default_bool_value))).astype(bool)
                else: # Se for numérico 0/1 ou outros
                    df_cleaned[col] = df_cleaned[col].astype(bool)
        except Exception as e:
            print(f"Não foi possível converter a coluna {col} para booleano diretamente: {e}. Verifique os valores.")


print("\nContagem de valores ausentes após tratamento geral:")
print(df_cleaned.isnull().sum())

print("\nPrimeiras 5 linhas do DataFrame limpo e preparado:")
print(df_cleaned.head())

print("\nInformações gerais do DataFrame limpo:")
df_cleaned.info() # Para verificar os tipos de dados após a limpeza

## 5. Configurações de Visualização
Definição de configurações globais para os gráficos que serão gerados, utilizando `seaborn` para o estilo e `matplotlib.pyplot` para o tamanho padrão das figuras e ajuste automático de layout.

In [ ]:
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 7)
# plt.rcParams['figure.autolayout'] = True # Causa UserWarning com tight_layout, pode ser removido se usar tight_layout()

_Nota: `figure.autolayout` = True pode, às vezes, entrar em conflito ou ser redundante com `plt.tight_layout()` usado posteriormente.
Pode ser comentado ou removido se `tight_layout()` for usado consistentemente._



## 6. Análise de Preferências Solicitadas
Visualização dos principais insights solicitados sobre as preferências dos estudantes:

 - Total de respostas por Linguagem de Programação Preferida.
 - Percentual de preferência por Horário de Estudo.
 - Formato de Conteúdo Principal Mais Popular.

In [ ]:

# Insight 1: Total de respostas por Linguagem de Programação Preferida
print("\n--- Insight 1: Linguagem de Programação Preferida ---")
linguagem_counts = df_cleaned['linguagem_preferida'].value_counts()
print(linguagem_counts)

plt.figure()
sns.barplot(x=linguagem_counts.index, y=linguagem_counts.values, palette="viridis", hue=linguagem_counts.index, legend=False)
plt.title('Total de Alunos por Linguagem de Programação Preferida', fontsize=16)
plt.xlabel('Linguagem de Programação', fontsize=14)
plt.ylabel('Número de Alunos', fontsize=14)
plt.xticks(rotation=45, ha='right')
plt.tight_layout() # Adicionado para melhor ajuste
plt.show()

# Insight 2: Percentual de preferência por Horário de Estudo
print("\n--- Insight 2: Horário de Estudo Preferido ---")
horario_counts = df_cleaned['horario_estudo'].value_counts()
horario_percentages = df_cleaned['horario_estudo'].value_counts(normalize=True) * 100
print(horario_percentages)

plt.figure(figsize=(10,8)) # Tamanho específico para gráfico de pizza
plt.pie(horario_counts, labels=horario_counts.index, autopct='%1.1f%%', startangle=140, colors=sns.color_palette("pastel"))
plt.title('Percentual de Preferência por Horário de Estudo', fontsize=16)
plt.axis('equal') # Assegura que o gráfico de pizza seja um círculo.
plt.tight_layout() # Adicionado para melhor ajuste
plt.show()

# Insight 3: Formato de Conteúdo Principal Mais Popular
print("\n--- Insight 3: Formato de Conteúdo Principal Preferido ---")
formato_principal_counts = df_cleaned['formato_conteudo_principal'].value_counts()
print(formato_principal_counts)
if not formato_principal_counts.empty: # Verifica se a série não está vazia
    print(f"O formato de conteúdo principal mais popular é: '{formato_principal_counts.idxmax()}' com {formato_principal_counts.max()} preferências.")

    plt.figure()
    sns.barplot(x=formato_principal_counts.index, y=formato_principal_counts.values, palette="coolwarm", hue=formato_principal_counts.index, legend=False)
    plt.title('Preferência por Formato de Conteúdo Principal', fontsize=16)
    plt.xlabel('Formato de Conteúdo', fontsize=14)
    plt.ylabel('Número de Alunos', fontsize=14)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout() # Adicionado para melhor ajuste
    plt.show()
else:
    print("Não há dados suficientes para determinar o formato de conteúdo principal mais popular.")

## 7. Gráficos Adicionais Explorando Outras Colunas
Geração de visualizações adicionais para explorar outras dimensões do conjunto de dados, incluindo:

 - Distribuição da Satisfação com o Curso.
 - Relação entre Horas de Estudo por Dia e Média Geral.
 - Contagem de Preferência por Área de Interesse (Top 10).
 - Percentual de Alunos que Utilizam a Biblioteca.
 - Distribuição da Média Geral por Semestre (Boxplot).

In [ ]:
# --- 6. Gráficos Adicionais Explorando Outras Colunas ---
print("\n--- Gráficos Adicionais ---")

# Gráfico 1: Distribuição da Satisfação com o Curso
if 'satisfacao_curso' in df_cleaned.columns:
    print("\nAnalisando 'satisfacao_curso'...")
    plt.figure()
    if pd.api.types.is_numeric_dtype(df_cleaned['satisfacao_curso']):
        sns.histplot(df_cleaned['satisfacao_curso'].dropna(), kde=True, bins=5) # Adicionado .dropna() para segurança
        plt.title('Distribuição da Satisfação com o Curso (1 a 5)', fontsize=16)
        plt.xlabel('Nível de Satisfação', fontsize=14)
        plt.ylabel('Número de Alunos', fontsize=14)
        plt.tight_layout() # Adicionado para melhor ajuste
        plt.show()
    else:
        print(f"'satisfacao_curso' (tipo: {df_cleaned['satisfacao_curso'].dtype}) não é numérica ou contém valores não convertidos. Verifique a etapa de limpeza.")
else:
    print("'satisfacao_curso' não encontrada.")

# Gráfico 2: Horas de Estudo por Dia vs. Média Geral
if 'horas_estudo_dia' in df_cleaned.columns and 'media_geral' in df_cleaned.columns:
    print("\nAnalisando 'horas_estudo_dia' vs 'media_geral'...")
    plt.figure()
    if pd.api.types.is_numeric_dtype(df_cleaned['horas_estudo_dia']) and pd.api.types.is_numeric_dtype(df_cleaned['media_geral']):
        sns.scatterplot(x='horas_estudo_dia', y='media_geral', data=df_cleaned, hue='satisfacao_curso', palette='coolwarm', alpha=0.7)
        plt.title('Horas de Estudo por Dia vs. Média Geral', fontsize=16)
        plt.xlabel('Horas de Estudo por Dia', fontsize=14)
        plt.ylabel('Média Geral', fontsize=14)
        plt.legend(title='Satisfação Curso')
        plt.tight_layout() # Adicionado para melhor ajuste
        plt.show()
    else:
        print(f"'horas_estudo_dia' (tipo: {df_cleaned['horas_estudo_dia'].dtype}) ou 'media_geral' (tipo: {df_cleaned['media_geral'].dtype}) não são numéricas. Verifique a limpeza.")
else:
    print("'horas_estudo_dia' ou 'media_geral' não encontradas.")


# Gráfico 3: Contagem de Preferência por Área de Interesse
if 'area_interesse' in df_cleaned.columns:
    print("\nAnalisando 'area_interesse'...")
    area_interesse_counts = df_cleaned['area_interesse'].value_counts().nlargest(10) # Top 10 para clareza
    if not area_interesse_counts.empty:
        plt.figure()
        sns.barplot(x=area_interesse_counts.index, y=area_interesse_counts.values, palette="cubehelix", hue=area_interesse_counts.index, legend=False)
        plt.title('Top 10 Áreas de Interesse dos Alunos', fontsize=16)
        plt.xlabel('Área de Interesse', fontsize=14)
        plt.ylabel('Número de Alunos', fontsize=14)
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout() # Adicionado para melhor ajuste
        plt.show()
    else:
        print("Não há dados para exibir sobre áreas de interesse.")
else:
    print("'area_interesse' não encontrada.")

# Gráfico 4: Uso da Biblioteca
if 'usa_biblioteca' in df_cleaned.columns:
    print("\nAnalisando 'usa_biblioteca'...")
    # Assegurar que a coluna seja tratada como categoria para value_counts
    # A conversão para string é uma forma robusta se o tipo booleano não for consistente.
    try:
        # Tenta converter para string e depois para booleano mapeado para melhor contagem
        map_bool_str = {True: 'Sim (True)', False: 'Não (False)', 'True': 'Sim (True)', 'False': 'Não (False)', 'Sim':'Sim (True)', 'Não':'Não (False)'}
        # O padrão é 'Não Informado' se o mapeamento falhar para algum valor inesperado
        processed_usa_biblioteca = df_cleaned['usa_biblioteca'].map(lambda x: map_bool_str.get(x, map_bool_str.get(str(x), 'Não Informado')))
        usa_biblioteca_counts = processed_usa_biblioteca.value_counts()

        if not usa_biblioteca_counts.empty:
            plt.figure(figsize=(8,6))
            plt.pie(usa_biblioteca_counts, labels=usa_biblioteca_counts.index, autopct='%1.1f%%', startangle=90, colors=['skyblue', 'lightcoral', 'lightgreen'])
            plt.title('Alunos que Utilizam a Biblioteca', fontsize=16)
            plt.axis('equal')
            plt.tight_layout() # Adicionado para melhor ajuste
            plt.show()
        else:
            print("Não há dados para exibir sobre o uso da biblioteca.")
    except Exception as e:
        print(f"Erro ao processar 'usa_biblioteca': {e}. Tipo da coluna: {df_cleaned['usa_biblioteca'].dtype}")
else:
    print("'usa_biblioteca' não encontrada.")


# Gráfico 5: Relação entre Semestre e Média Geral (Boxplot)
if 'semestre' in df_cleaned.columns and 'media_geral' in df_cleaned.columns:
    print("\nAnalisando 'semestre' vs 'media_geral'...")
    plt.figure(figsize=(14, 8))
    
    # Assegura que 'semestre' seja numérico para ordenação correta e depois convertido para string/categoria para o boxplot
    # Tenta converter 'semestre' para numérico, tratando erros
    df_cleaned['semestre_numeric'] = pd.to_numeric(df_cleaned['semestre'], errors='coerce')
    df_temp = df_cleaned.dropna(subset=['semestre_numeric', 'media_geral']) # Remove NaNs criados por 'coerce' ou já existentes
    
    if not df_temp.empty:
        # Ordena os valores únicos de semestre numericamente
        semestre_order = sorted(df_temp['semestre_numeric'].unique())
        # Converte para string para o boxplot usar como categorias ordenadas
        semestre_order_str = [str(int(s)) for s in semestre_order]
        
        df_temp['semestre_cat_ordered'] = pd.Categorical(df_temp['semestre_numeric'].astype(int).astype(str), categories=semestre_order_str, ordered=True)

        sns.boxplot(x='semestre_cat_ordered', y='media_geral', data=df_temp, palette="pastel") # Usando a coluna ordenada
        plt.title('Distribuição da Média Geral por Semestre', fontsize=16)
        plt.xlabel('Semestre', fontsize=14)
        plt.ylabel('Média Geral', fontsize=14)
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout() # Adicionado para melhor ajuste
        plt.show()
    else:
        print("Não há dados numéricos válidos suficientes em 'semestre' ou 'media_geral' para gerar o boxplot.")
else:
    print("'semestre' ou 'media_geral' não encontradas.")

# 8. Conclusão da Análise
Este bloco finaliza a execução da análise exploratória de dados.



In [ ]:
print("\n--- Fim da Análise ---")

# Conclusão Resumida Sobre o Processo de Análise do Script
Ao processar e estruturar o script em formato de notebook, observei um pipeline de análise de dados eficaz e bem definido.

O script demonstrou um fluxo de trabalho lógico, desde a importação de bibliotecas e carregamento dos dados, passando por uma inspeção inicial completa, até uma etapa de limpeza e preparação de dados robusta. Esta limpeza destacou-se pela atenção aos detalhes, com tratamento diferenciado para valores ausentes (NaN) conforme o tipo de dado e conversões de tipo cuidadosas, como a de colunas para booleano com tratamento de exceções.

A abordagem analítica e de visualização foi direta e apropriada, utilizando gráficos como barras, pizza, histogramas, scatterplots e boxplots para extrair tanto os insights solicitados sobre preferências estudantis quanto para realizar explorações adicionais em outras variáveis. A inclusão de verificações de robustez, como checar a existência de colunas antes de usá-las, também foi uma prática positiva observada.

Em suma, a análise deste script reforçou a compreensão de um processo completo e bem executado para transformar dados brutos em insights visuais, enfrentando de forma competente os desafios comuns na preparação e exploração de dados.






